In [5]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 46 µs


In [6]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

print df.shape, test.shape

(69713, 22)
(30037, 21)
CPU times: user 57.1 s, sys: 37.8 s, total: 1min 34s
Wall time: 1min 34s


In [10]:
%%time
np.save('yesterday_train.npy', df)
np.save('yesterday_test.npy', test)

CPU times: user 54 ms, sys: 7.06 s, total: 7.11 s
Wall time: 11.5 s


In [13]:
!ls -sh|grep yes

8.4G yesterday_test.npy
 20G yesterday_train.npy


In [7]:
type(test)

numpy.ndarray

In [10]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        61G        64G       2.6M       9.3M        75M
-/+ buffers/cache:        61G        64G
Swap:         4.0G       154M       3.8G


### from here

In [2]:
%%time
train = np.load('yesterday_train.npy')
test = np.load('yesterday_test.npy')
y = np.load('y.npy')

CPU times: user 0 ns, sys: 9.13 s, total: 9.13 s
Wall time: 15.5 s


In [3]:
from sklearn.feature_selection import RFE

In [11]:
%%time
model=RandomForestClassifier(random_state=7, n_estimators=600, min_samples_split=5, n_jobs=-1)
rfe = RFE(model, train.shape[1]*2/3, step=0.1, verbose=1)
rfe = rfe.fit(train, y)
# summarize the selection of the attributes
print(rfe.support_)
# print(rfe.ranking_)
index2=rfe.support_

Fitting estimator with 37416 features.
Fitting estimator with 33675 features.
Fitting estimator with 29934 features.
Fitting estimator with 26193 features.
[ True  True  True ...  True  True  True]
CPU times: user 1h 12min 9s, sys: 24.8 s, total: 1h 12min 34s
Wall time: 4min 16s


In [20]:
%%time
print train.shape, test.shape
train, test = train[:,index2], test[:, index2]
print train.shape, test.shape

(69713, 24944) (30037, 24944)


In [21]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state  = 7)
print m(X_train, y_train, X_test, y_test)

[(55770, 24944), (55770,), (13943, 24944), (13943,)]
CPU times: user 18.2 s, sys: 7.55 s, total: 25.8 s
Wall time: 25.8 s


In [ ]:
%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

In [ ]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.05,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [8]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=10, 
                verbose_eval=3,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.788639	test-auc:0.793015
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 30 rounds.
[3]	train-auc:0.814173	test-auc:0.799279
[6]	train-auc:0.831896	test-auc:0.805355
[9]	train-auc:0.862311	test-auc:0.815797
[12]	train-auc:0.867843	test-auc:0.816219
[15]	train-auc:0.873361	test-auc:0.817897
[18]	train-auc:0.881048	test-auc:0.816382
[21]	train-auc:0.886893	test-auc:0.816326
[24]	train-auc:0.893352	test-auc:0.81897
[27]	train-auc:0.8958	test-auc:0.819903
[30]	train-auc:0.901705	test-auc:0.820251
[33]	train-auc:0.906898	test-auc:0.8211
[36]	train-auc:0.910569	test-auc:0.82212
[39]	train-auc:0.916876	test-auc:0.825781
[42]	train-auc:0.923367	test-auc:0.828004
[45]	train-auc:0.931784	test-auc:0.829572
[48]	train-auc:0.937186	test-auc:0.83181
[51]	train-auc:0.941429	test-auc:0.832547
[54]	train-auc:0.945593	test-auc:0.834497
[57]	train-auc:0.949939	test-auc:0.836716
[60]	train-auc:0.953861	test-a

'!!! used 808.05 s'

In [13]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 14.9 s, sys: 7.02 s, total: 21.9 s
Wall time: 21.9 s


In [14]:
%%time
predb = clf.predict(dtest)


CPU times: user 4.3 s, sys: 1 ms, total: 4.3 s
Wall time: 268 ms


___

In [19]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [21]:
submit.to_csv('xgb1.csv',index=False)

In [22]:
clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba